In [103]:
import gist ### https://github.com/tuttieee/lear-gist-python

from PIL import Image
import imageio
from IPython.display import display, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, roc_auc_score


#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Parameters

In [4]:
# parameters
BASE_FOLDER = os.getenv('USGS_FOLDER')
MFP_FOLDER = os.getenv('MFP_FOLDER')
MFP_FOLDER

'/Users/peterweber/Google Drive/MFP - Satellogic/'

### Load labeled images 

In [34]:
labels_csv = os.listdir(MFP_FOLDER + "labels")
labels_csv = [label_csv for label_csv in labels_csv if label_csv.startswith("labels")]
cols = ['filename', 'label']

df_labeled = pd.DataFrame(columns = cols)
for fname_csv in labels_csv:
    df = pd.read_csv(MFP_FOLDER + "labels/" + fname_csv)
    df_labeled = pd.concat([df_labeled[cols], df[cols]], axis=0)

In [40]:
images = []
for fname in df_labeled.filename:
    imarray = np.asarray(Image.open(MFP_FOLDER + "processedIMG/"+ fname))[:,:,:3]
    images.append(imarray)

In [41]:
df_labeled['imarray'] = images

In [42]:
df_labeled.head()

,filename,label,imarray
0,m_4709143_nw_15_1_20171010_x2560_y1024.png,0,"[[[54, 69, 48], [69, 106, 58], [64, 83, 56], [..."
1,m_4309736_nw_14_1_20140813_x5632_y4096.png,1,"[[[55, 75, 66], [57, 75, 67], [56, 74, 66], [5..."
2,m_4709143_nw_15_1_20171010_x2048_y3584.png,0,"[[[78, 81, 65], [53, 54, 50], [52, 60, 51], [9..."
3,m_2608164_sw_17_1_20171210_x2048_y2560.png,0,"[[[112, 138, 103], [132, 157, 118], [110, 132,..."
4,m_3409456_ne_15_1_20170914_x0_y512.png,1,"[[[164, 159, 117], [158, 155, 115], [162, 157,..."


In [56]:
# Generate feature vector with gist
feature_vector = []
for image in df_labeled["imarray"].tolist():
    feature_vector.append(gist.extract(image, nblocks=1, orientations_per_scale=(8, 8, 4)))
    
feature_matrix = np.array(feature_vector)

### Classification on binary label

In [96]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, df_labeled.label.tolist(), test_size = 0.2)

In [97]:
clf = LinearSVC()# for multi class labels use multi_class="crammer_singer"

In [98]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [99]:
y_pred = clf.predict(X_test)

In [100]:
accuracy_score(y_test, y_pred)

0.7307692307692307

In [101]:
roc_auc

array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0])